In [ ]:
import os
from bs4 import BeautifulSoup
import bs4
import cv2
import random
import time

def path2img(img_num):
    img_pad = str(img_num).zfill(6)
    return f"frame_{img_pad}.PNG"
            
def load_annotations(annotation_path: str):
    r"""
    annotation_path: str = "D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_02_05_18_52_05-cvat for video 1.1"
    """
    path=f"{annotation_path}\\annotations.xml"
    with open(path, 'r') as f:
        data = f.read()
    Bs_data = BeautifulSoup(data, "xml")
    metadata = Bs_data.find('meta').find("task").find("segments")
    # Get the annotations frame count
    start_frame = int(metadata.find("start").text)
    stop_frame = int(metadata.find("stop").text)
    # The annotations is within track property
    box_annotations = Bs_data.find_all('track')
    # Check if there are any annotations of fish, else return empty box annotations
    if box_annotations:
        # Only get box elements
        map_frame_to_index = {}
        all_boxes = box_annotations[0].find_all('box')
        for i in range(1, len(box_annotations)):
            all_boxes += box_annotations[i].find_all('box')
        for j, box in enumerate(all_boxes):
            #print(str(box["frame"]))
            if box["frame"] in map_frame_to_index:
                map_frame_to_index[box["frame"]] += [j]
            else:
                map_frame_to_index[box["frame"]] = [j]
        return all_boxes, map_frame_to_index, (start_frame, stop_frame)
    return "", {}, (start_frame, stop_frame)


def cvat2yolov2(dataset_train_folders: list[str], dataset_test_folders: list[str], dataset_valid_folders: list[str], dataset_output_folder: str, imagetype: str =".PNG"):
    """
    Read annotations.txt CVAT and writes to a .txt file for each frame with YOLO annotation
    Expects a datasetfolder which includes the following
    {dataset_folder}/annotations.txt
    {dataset_folder}/images/frame_000000.PNG
    """
    # Create output directory structure
    if not os.path.exists(dataset_output_folder):
        os.mkdir(dataset_output_folder)
    sub_folders = ["test","train","valid"]
    sub_sub_folders = ["images", "labels"]
    for sub_folder in sub_folders:
        sub_path = os.path.join(dataset_output_folder,sub_folder)
        if not os.path.exists(sub_path):
            os.mkdir(sub_path)
        for sub_sub in sub_sub_folders:
            sub_sub_path = os.path.join(sub_path, sub_sub)
            if not os.path.exists(sub_sub_path):
                os.mkdir(sub_sub_path)
    # Loop through 3 dataset_folders to be added to test, train, valid
    dataset_output_sub_folder = [os.path.join(dataset_output_folder,sub_folder) for sub_folder in sub_folders]

    def write_yolo_format(dataset_folder: str, dataset_output_sub_folder: str, img_start_count: int):
        # for dataset_folder, dataset_output_sub_folder in zip(dataset_folders, dataset_output_sub_folder):
        print(dataset_folder, dataset_output_sub_folder)
        boxes, map2indices, frames = load_annotations(dataset_folder)
        start_idx, stop_idx = frames
        for idx, i in enumerate(range(start_idx, stop_idx+1)):  
            img_shortpath = path2img(i)
            img_out_shortpath = path2img(img_start_count + idx)
            img_path = os.path.join(dataset_folder, "images", img_shortpath)
            annotation_out_path = os.path.join(dataset_output_sub_folder, "labels", img_out_shortpath[0:-4]+".txt")
            annotation_data = ""
            # Save image in output dataset
            im = cv2.imread(img_path)
            img_out_path = os.path.join(dataset_output_sub_folder, "images", img_out_shortpath)
            #print(f"Will save to {img_out_path}")
            cv2.imwrite(img_out_path, im)

            # Compute and save annotations to output dataset
            if str(i) in map2indices:
                imshape = im.shape
                im_width = imshape[1]
                im_height = imshape[0]
                for indicie in map2indices[str(i)]:
                    box = boxes[indicie]
                    x = float(box['xtl'])
                    y = float(box['ytl'])
                    x2 = float(box['xbr'])
                    y2 = float(box['ybr'])
                    # Compute Yolo annotation from x1,y1,x2,y2 values
                    yolo_width = (x2-x) / im_width
                    yolo_height = (y2-y) / im_height
                    yolo_x = x/im_width+yolo_width/2
                    yolo_y = y/im_height+yolo_height/2
                    # Convert to integers
                    annotation_data+=f"0 {str(yolo_x)} {str(yolo_y)} {str(yolo_width)} {str(yolo_height)}\n"
                    #print(annotation_data.rstrip("\n"))
                    #print(box)     
            else:
                pass
            with open(annotation_out_path, "w") as fp:
                #print(annotation_out_path)
                fp.write(annotation_data)
        return idx

    # Remember to add a counter of the images where you knwo where to start the count. Since all the datasets have different frames, this has to be incremated.
    # Train
    start_count = 0
    
    #for dataset_folder in dataset_train_folders:
    #    start_count += write_yolo_format(dataset_folder, os.path.join(dataset_output_folder, "train"), start_count)
    # Test
    start_count = 0
    for dataset_folder in dataset_test_folders:
        start_count += write_yolo_format(dataset_folder, os.path.join(dataset_output_folder, "test"), start_count)
    # Valid
    start_count = 0
    for dataset_folder in dataset_valid_folders:
        start_count += write_yolo_format(dataset_folder, os.path.join(dataset_output_folder, "valid"), start_count)

In [ ]:
rootpath = r"D:\Dataset\FishDetection\Annotation"
directories = os.listdir(r"D:\Dataset\FishDetection\Annotation")
db_paths = [os.path.join(rootpath, direct) for direct in directories]
    
random.shuffle(db_paths)
train = int(len(db_paths) * 0.8)
test = train + int((len(db_paths) - train) / 2)
valid = len(db_paths)

train_set = db_paths[:train]
test_set = db_paths[train:test]
valid_set = db_paths[test:]

output_path = r"D:\Dataset\FishDetection2Yolo"
cvat2yolov2(train_set, test_set, valid_set, output_path)

D:\Dataset\FishDetection\Annotation\task_20230717035253_20230717035323_3.mp4_dataset_2024_02_26_06_56_19_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\train
D:\Dataset\FishDetection\Annotation\task_20230718033643_20230718033654_3.mp4_dataset_2024_02_26_06_58_59_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\train
D:\Dataset\FishDetection\Annotation\task_20230717033504_20230717033512_3.mp4_dataset_2024_02_26_06_55_53_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\train


In [41]:
cvat2yolov2(train_set, test_set, valid_set, output_path)

D:\Dataset\FishDetection\Annotation\task_20230717034948_20230717035031_3.mp4_dataset_2024_02_26_06_55_57_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\test
D:\Dataset\FishDetection\Annotation\task_20230717035235_20230717035253_3.mp4_dataset_2024_02_26_06_56_17_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\test
D:\Dataset\FishDetection\Annotation\task_20230718085420_20230718085435_3.mp4_dataset_2024_02_28_10_02_13_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\test
D:\Dataset\FishDetection\Annotation\task_fish_detection-2024_02_01_18_55_41-cvat for video 1.1 D:\Dataset\FishDetection2Yolo\test
D:\Dataset\FishDetection\Annotation\task_20230717035359_20230717035412_3.mp4_dataset_2024_02_26_06_56_27_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\test
D:\Dataset\FishDetection\Annotation\task_20230718030438_20230718030447_3.mp4_dataset_2024_02_26_06_58_24_cvat for video 1.1 D:\Dataset\FishDetection2Yolo\test
D:\Dataset\FishDetection\Annotation\task_20230717035521_20230717035530_3.mp

In [102]:
dataset_folder = r"D:\Dataset\FishDetection\Annotation\task_20230718031518_20230718031525_3.mp4_dataset_2024_02_26_06_58_29_cvat for video 1.1"
boxes, map2indices, frames = load_annotations(dataset_folder)
start_idx, stop_idx = frames
images = [ os.path.join(dataset_folder,"images",img) for img in os.listdir(os.path.join(dataset_folder,"images")) if img[-3:].lower() == "png" or img[-3:].lower() == "jpg"]
annotation_path = img_path.rstrip(imagetype)+".txt"

In [31]:
train_set

['D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_02_05_07_43_53-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_02_05_18_22_08-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717035112_20230717035132_3.mp4_dataset_2024_02_26_06_56_07_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717035147_20230717035212_3.mp4_dataset_2024_02_26_06_56_12_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717035438_20230717035509_3.mp4_dataset_2024_02_26_06_56_47_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717033352_20230717033359_3.mp4_dataset_2024_02_26_06_55_38_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230716013052_20230716013101_3.mp4_dataset_2024_02_24_12_58_25_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_02_05_18_29_05-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\A

In [33]:
test_set

['D:\\Dataset\\FishDetection\\Annotation\\task_20230717034948_20230717035031_3.mp4_dataset_2024_02_26_06_55_57_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717035235_20230717035253_3.mp4_dataset_2024_02_26_06_56_17_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230718085420_20230718085435_3.mp4_dataset_2024_02_28_10_02_13_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_02_01_18_55_41-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717035359_20230717035412_3.mp4_dataset_2024_02_26_06_56_27_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230718030438_20230718030447_3.mp4_dataset_2024_02_26_06_58_24_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717035521_20230717035530_3.mp4_dataset_2024_02_26_06_56_50_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230716011538_20230716011547_3.mp4_dataset_2024_02_24_12_

In [35]:
valid_set

['D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_01_28_18_21_42-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717033336_20230717033353_3.mp4_dataset_2024_02_26_06_55_36_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_01_28_18_28_06-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717035607_20230717035616_3.mp4_dataset_2024_02_26_06_56_56_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_01_28_18_41_54-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230717011703_20230717011711_3.mp4_dataset_2024_02_26_06_55_16_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_20230718031518_20230718031525_3.mp4_dataset_2024_02_26_06_58_29_cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_fish_detection-2024_02_01_18_48_30-cvat for video 1.1',
 'D:\\Dataset\\FishDetection\\Annotation\\task_2023071703535

In [ ]:
!yolo val model="./runs/runs/detect/train13/weights/best.pt" data="D:\Dataset\FishDetection2Yolo\data.yaml"

In [ ]:
start_time = time.time()
!..\py39\Scripts\yolo.exe detect train data=D:/Dataset/FishDetection2Yolo/data.yaml model=./runs/detect/train13/weights/best.pt epochs=60 imgsz=640
stop_time = time.time()
print(f"Training took: {stop_time-start_time}s")

In [ ]:
passed_hours = (stop_time-start_time)/3600
print(f"{passed_hours} hours")